# Notebook to create datasets to create DEALA activities for transport

## 1. Initalization of project

## 1.1 load python packages

<p style="font-size:18px">
To load all required Python modules and packages.
</p>

In [4]:
# initializatiin of packages
import brightway2 as bw
import os
import matplotlib.pyplot as plt
import numpy as np
from bw2io import *
from tqdm import *
import toml
import requests
import gzip
import pandas as pd
from io import BytesIO
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import json
import uuid
import deala as de
from deala import deala_io
deala_io_instance = deala_io()


## 1.2 Set up project

<p style="font-size:12px">
The following aspects are defined in this substep:
</p>

<ol style="font-size:12px; padding-left: 20px">
  <li><code>set_current</code>: Defines the name of the project and loads all associated data sets. In this assessment, the project is named <code>regiopremise</code>.</li><br><br>
  <li><code>Client_ID</code>: Client ID for UN trade & development Data Hub</li><br><br>
  <li><code>API_key</code>: API key for UN trade & development Data Hub</li><br><br>
  <li><code>path_transport_data</code>: Path were data of UN trade and development are stored</li><br><br>
  <li><code>dict_countries</code>: Defines the countries that produce the pans or represent the comsumer market.</li><br><br>


In [2]:
#Create/load project
project_name ='regiopremise_test_small2'
bw.projects.set_current(project_name)

# Client ID and API key for UN trade & development Data Hub
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop", "key_transport_cost.toml")
config = toml.load(desktop_path)
CLIENT_ID = config['auth']['CLIENT_ID']
API_KEY = config['auth']['API_KEY']

# Definition path to store downloaded data 
# Get the directory of the deala package
directory_deala = os.path.dirname(de.__file__)
path_transport_data=os.path.join(directory_deala, 'files', 'linking')

# Definition path to store transport DEALA activity json files
path_transport_json=os.path.join(directory_deala, 'files', 'DEALA_activities')

#Definition of countries for production activities that should be regionalized and for the consumer markets
dict_countries={
    'China': 'CN',
    'France': 'FR',
    'Germany': 'DE',
    'India': 'IN',
    'Italy': 'IT',
    'United States of America': 'US',
    'Turkey': 'TR',
    'Netherlands': 'NL',
    'Poland': 'PL',
    'Belgium': 'BE',
    'Czechia': 'CZ',
}

## 2. Create data sets for transport in DEALA

## 2.1 Load all data from [UN trade & development Data Hub](https://unctadstat.unctad.org/datacentre/dataviewer/US.TransportCosts) via API

In [3]:
# ============================================================================
# API CONFIGURATION
# ============================================================================
API_URL = "https://unctadstat-user-api.unctad.org/US.TransportCosts/cur/Facts?culture=en"
CLIENT_ID = CLIENT_ID
API_KEY = API_KEY
HEADERS = {"ClientId": CLIENT_ID, "ClientSecret": API_KEY}

# ============================================================================
# ORIGINS and TRANSPORT MODES
# ============================================================================
origins = [
    "001",  # 🌍 World (global origin)
    '0001', '004','008','012','020','024','028','032', '050','051','533','036','040','031','044','048','052','112','056',
    '084','204','060','068','070','072','076','096','100', '124', '854','108','132','116','120','136','140','152','156',
    '344','446','170','174','178','180','188','384','191','196','203','208','214','218','818','222','233','748',
    '231','242','246','251','258','266', '270','268','276','288','300','304','308','320','324','328','340','348','352',
    '356','360','364','372','376','380','388','392','400','398','404','296','410','414','417','418','428','422',
    '426','434','440','442','450','454','458','462','466','470','478','480', '484','498','496','499','500','504','508',
    '104','516','524','528','554','558','562','566','807','579','512','586','585','591','600','604','608','616',
    '620','634','642','643','646','659','662','670','882','678','682','686','688','690','694','702','703','705',
    '090','724','144','275','710', '729','740','752','757','762','834','764','626','768','780','788','792','800','804',
    '784','926','842','858','860','704','887','894','716', '178'
]

transport_modes = {
    "10": "Air",
    "21": "Sea",
    "31": "Railway",
    "32": "Road"
}

# ============================================================================
# Helper function to fetch data from the API
# ============================================================================
def fetch_data(origin_code, transport_code):
    filter_query = (
        f"TransportMode/Code eq '{transport_code}' and "
        f"Origin/Code eq '{origin_code}' and "
        f"Product/Code eq 'TOTAL' and "
        f"Year in (2016,2017,2018,2019,2020,2021)"
    )

    form_data = {
        "$select": "Origin/Label,Destination/Label,TransportMode/Label,Year,Transport_cost_intensity_in_US_per_tonkm_Value",
        "$filter": filter_query,
        "$orderby": "Destination/Order asc ,Year asc",
        "$compute": "round(M1991/Value div 1, 3) as Transport_cost_intensity_in_US_per_tonkm_Value",
        "$format": "csv",
        "compress": "gz"
    }

    try:
        r = requests.post(API_URL, data=form_data, headers=HEADERS, timeout=90)
        r.raise_for_status()

        with gzip.open(BytesIO(r.content), "rt", encoding="utf-8") as f:
            df = pd.read_csv(f)

        if df.empty:
            return pd.DataFrame()

        # Normalize column names
        df.columns = [c.strip().replace("/", "_").replace(".", "_") for c in df.columns]

        df["Origin_Code"] = origin_code
        df["TransportMode_Code"] = transport_code
        df["TransportMode_Label"] = transport_modes.get(transport_code, "Unknown")

        return df

    except Exception as e:
        print(f"⚠️ Error for Origin {origin_code}, Mode {transport_code}: {e}")
        return pd.DataFrame()

# ============================================================================
# Parallel data fetching
# ============================================================================
tasks = [(o, m) for o in origins for m in transport_modes.keys()]
print(f"Starting {len(tasks)} parallel requests...")

all_data = []
start = time.time()

with ThreadPoolExecutor(max_workers=8) as pool:
    futures = {pool.submit(fetch_data, o, m): (o, m) for o, m in tasks}
    for i, fut in enumerate(as_completed(futures), 1):
        df = fut.result()
        if not df.empty:
            all_data.append(df)
        if i % 10 == 0:
            print(f"{i}/{len(futures)} completed")

print(f"Finished in {round(time.time() - start, 1)} seconds")

# ============================================================================
# Check for existence of "World" data
# ============================================================================
if all_data:
    combined_test_df = pd.concat(all_data, ignore_index=True)
    origins_in_data = combined_test_df["Origin_Label"].unique().tolist()

    if not any("World" in str(x) for x in origins_in_data):
        print("⚠️ No 'World' entries found as Origin – starting additional request for Origin/Code='001'...")

        world_tasks = [("001", m) for m in transport_modes.keys()]
        world_data = []

        with ThreadPoolExecutor(max_workers=4) as pool:
            futures = {pool.submit(fetch_data, o, m): (o, m) for o, m in world_tasks}
            for i, fut in enumerate(as_completed(futures), 1):
                df = fut.result()
                if not df.empty:
                    world_data.append(df)
                if i % 2 == 0:
                    print(f"🌍 Additional request progress: {i}/{len(world_tasks)}")

        # Try code "0" if "001" gives no data
        if not world_data:
            print("⚠️ No data with code '001' – trying code '0'...")
            world_tasks_zero = [("0", m) for m in transport_modes.keys()]
            with ThreadPoolExecutor(max_workers=4) as pool:
                futures = {pool.submit(fetch_data, o, m): (o, m) for o, m in world_tasks_zero}
                for i, fut in enumerate(as_completed(futures), 1):
                    df = fut.result()
                    if not df.empty:
                        world_data.append(df)
                    if i % 2 == 0:
                        print(f"🌍 Additional request (0) progress: {i}/{len(world_tasks_zero)}")

        if world_data:
            print(f"✅ Additional request successful: {len(world_data)} datasets found with 'World' as Origin.")
            all_data.extend(world_data)
        else:
            print("⚠️ No 'World' data found, even after retrying with (001/0).")
    else:
        print("✅ 'World' Origin data already present.")
else:
    print("⚠️ No data loaded — skipping 'World' check.")

# ============================================================================
# COMBINE AND PIVOT DATA
# ============================================================================
if not all_data:
    raise ValueError("No data retrieved")

full = pd.concat(all_data, ignore_index=True)

# Harmonize column names
rename_map = {
    "Origin_Label": "Origin.Label",
    "Destination_Label": "Destination.Label",
    "TransportMode_Label": "TransportMode.Label",
    "Transport_cost_intensity_in_US_per_tonkm_Value": "TransportCost"
}
for old, new in rename_map.items():
    if old in full.columns:
        full.rename(columns={old: new}, inplace=True)
    elif new not in full.columns:
        full[new] = "Unknown"

pivot = (
    full.pivot_table(
        index=["Origin_Code", "Origin.Label", "Destination.Label", "TransportMode.Label"],
        columns="Year",
        values="TransportCost",
        aggfunc="mean"
    )
    .reset_index()
)
pivot.columns.name = None
pivot.rename(columns=lambda c: f"{c}" if isinstance(c, int) else c, inplace=True)

# Save to CSV
pivot.to_csv(os.path.join(path_transport_data, "unctad_transport_costs_fixed_pivot.csv"), index=False)
print("✅ File saved: unctad_transport_costs_fixed_pivot.csv")

# Output short check
print("🌍 Origin labels containing 'World':", [x for x in full["Origin.Label"].unique() if "World" in str(x)])
print(pivot.head(10))


Starting 712 parallel requests...
10/712 completed
20/712 completed
30/712 completed
40/712 completed
50/712 completed
60/712 completed
70/712 completed
80/712 completed
90/712 completed
100/712 completed
110/712 completed
120/712 completed
130/712 completed
140/712 completed
150/712 completed
160/712 completed
170/712 completed
180/712 completed
190/712 completed
200/712 completed
210/712 completed
220/712 completed
230/712 completed
240/712 completed
250/712 completed
260/712 completed
270/712 completed
280/712 completed
290/712 completed
300/712 completed
310/712 completed
320/712 completed
330/712 completed
340/712 completed
350/712 completed
360/712 completed
370/712 completed
380/712 completed
390/712 completed
400/712 completed
410/712 completed
420/712 completed
430/712 completed
440/712 completed
450/712 completed
460/712 completed
470/712 completed
480/712 completed
490/712 completed
500/712 completed
510/712 completed
520/712 completed
530/712 completed
540/712 completed
550

## 2.2 Create json file for transport DEALA activities

In [6]:
# ============================================================================
# 0. Prepare country selection
# ============================================================================
# Read countries from input databases and include those defined manually

list_countries = []
for db in bw.databases:
    if "input" in db:
        for act in bw.Database(db):
            country = act['location']
            if country not in list_countries:
                list_countries.append(country)

# Add countries from dict_countries if not already included
for country in dict_countries.keys():
    if dict_countries[country] not in list_countries:
        list_countries.append(dict_countries[country])

print(f"🌍 Using {len(list_countries)} countries: {list_countries}")

# ============================================================================
# 1. Load input data
# ============================================================================
pivot = pd.read_csv(os.path.join(path_transport_data, "unctad_transport_costs_fixed_pivot.csv"))
countries = pd.read_excel(
    os.path.join(path_transport_data, "countries.xlsx"),
    keep_default_na=False,
    na_values=[],
    dtype=str
)

records = []

# ============================================================================
# 2. Main loop over all rows from the UNCTAD pivot file
# ============================================================================
for _, row in pivot.iterrows():
    # Only 2019 is used here, but structure allows for extending year range
    years = [str(y) for y in range(2019, 2020) if str(y) in pivot.columns]
    costs = {str(y): row[str(y)] for y in years}

    origin_label = row["Origin.Label"]
    dest_label = row["Destination.Label"]

    # Helper function: map country/region labels to ISO and REMIND codes
    def get_country_info(label):
        if label in countries["country"].values:
            iso = countries.loc[countries["country"] == label, "alpha-2"].values[0]
            remind = countries.loc[countries["country"] == label, "remind"].values[0]
        elif label in countries["region"].values:
            iso = countries.loc[countries["region"] == label, "alpha-2"].values[0]
            remind = countries.loc[countries["region"] == label, "remind"].values[0]
        elif label in countries["sub-region"].values:
            iso = countries.loc[countries["sub-region"] == label, "alpha-2"].values[0]
            remind = countries.loc[countries["sub-region"] == label, "remind"].values[0]
        else:
            return None, None
        return iso, remind

    origin_iso, origin_remind = get_country_info(origin_label)
    dest_iso, dest_remind = get_country_info(dest_label)

    if not origin_iso or not dest_iso:
        continue
    if origin_iso not in list_countries or dest_iso not in list_countries:
        continue

    if pd.notna(costs["2019"]):
        cost_value = float(costs["2019"])

        # --- Standard record: Origin → Destination ---
        records.append({
            "Country": origin_label,
            "ISO-3166-1 ALPHA-2": origin_iso,
            "REMIND Region": origin_remind,
            "Sector": "Transport",
            "Type": f"Transport, {row['TransportMode.Label']}, from {origin_iso} to {dest_iso}",
            "Costs per unit [USD/unit]": cost_value,
            "Unit": "tkm",
            "Base Year": 2019,
            "Identifier": "transport",
            "Years": 0,
            "Description": (
                f"This dataset describes the average transport cost per ton-kilometer "
                f"via {row['TransportMode.Label']} from {origin_label} to {dest_label} "
                f"(2019 data, source: UNCTAD)."
            ),
            "Code": str(uuid.uuid4().hex)
        })

        # --- Additional record: Global (GLO) → Origin (if destination is World) ---
        if dest_label == "World":
            records.append({
                "Country": "GLO",
                "ISO-3166-1 ALPHA-2": "GLO",
                "REMIND Region": "GLO",
                "Sector": "Transport",
                "Type": f"Transport, {row['TransportMode.Label']}, from GLO to {origin_iso}",
                "Costs per unit [USD/unit]": cost_value,
                "Unit": "tkm",
                "Base Year": 2019,
                "Identifier": "transport",
                "Years": 0,
                "Description": (
                    f"Estimated transport cost from GLO to {origin_label}, "
                    f"based on the inverse of {origin_label} to World for "
                    f"{row['TransportMode.Label']} (2019, UNCTAD)."
                ),
                "Code": str(uuid.uuid4().hex)
            })

# ============================================================================
# 3. Add domestic transport entries (e.g., DE → DE)
# ============================================================================
df_costs = pd.DataFrame(records)
intra_added = 0

for mode in df_costs["Type"].apply(lambda x: x.split(",")[1].strip()).unique():
    for country in list_countries:
        subset = df_costs[df_costs["Type"].str.contains(mode)]
        subset = subset[subset["Type"].str.contains(country)]
        if subset.empty:
            continue

        min_cost = subset["Costs per unit [USD/unit]"].replace(0, np.nan).min()
        if pd.notna(min_cost) and min_cost > 0:
            records.append({
                "Country": country,
                "ISO-3166-1 ALPHA-2": country,
                "REMIND Region": countries.loc[countries["alpha-2"] == country, "remind"].values[0],
                "Sector": "Transport",
                "Type": f"Transport, {mode}, from {country} to {country}",
                "Costs per unit [USD/unit]": round(min_cost, 4),
                "Unit": "tkm",
                "Base Year": 2019,
                "Identifier": "transport",
                "Years": 0,
                "Description": (
                    f"Estimated domestic transport within {country} via {mode}, "
                    f"using the minimum observed cost among all international relations."
                ),
                "Code": str(uuid.uuid4().hex)
            })
            intra_added += 1

print(f"✅ Added {intra_added} domestic transport entries.")
print(f"✅ Total records: {len(records)}")

# ============================================================================
# 4. Save to JSON
# ============================================================================
with open(os.path.join(path_transport_json, "transport.json"), "w", encoding="utf-8") as f:
    json.dump(records, f, ensure_ascii=False, indent=2)

print("✅ JSON file created: transport.json")


🌍 Using 26 countries: ['GLO', 'BR', 'PL', 'US', 'RU', 'MX', 'CN', 'CD', 'RoW', 'NL', 'RER', 'JP', 'IN', 'IR', 'FR', 'DE', 'CZ', 'TR', 'CA', 'IT', 'DZ', 'RO', 'AE', 'BE', 'CL', 'KR']
✅ Added 100 domestic transport entries.
✅ Total records: 2139
✅ JSON file created: transport.json
